In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect
import sys

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir el uso y entendimiento de las funciones definidas para resolver un Sistema de Ecuaciones Lineales (SEL) a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
#Código "crear_matriz_aleatoria"
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstanc

In [3]:
#ayuda "crear_matriz_aleatoria"
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
#código "house"
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
     

In [5]:
#ayuda "house"
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
#código "matriz_auxiliar_Arv"
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A,ind_singular=False):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columna

In [7]:
#Código "matriz_auxiliar_Arv"
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A, ind_singular=False)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogona

In [8]:
#Código "matriz_Q_R"
lines=inspect.getsource(fQR.matriz_Q_R)
print(lines)

def matriz_Q_R(A,ind_singular=False):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)

    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    #si ind_singular es True, entonces no se devuelve la matriz R ni Q
    #esto se plantea así como un sistema de contról, para evitar comprobar que las matrices
    #sean singulares (y no tengan una única solución) mediante el determinante, que
    #involucra un alto costo computacional para matrices de tamaño representativo
    Arv=matriz_auxiliar_Arv(A,ind_singular)

    if Arv is None:
        return None, None
 

In [9]:
#Ayuda "matriz_Q_R"
help(fQR.matriz_Q_R)

Help on function matriz_Q_R in module funciones_factorizacion_QR:

matriz_Q_R(A, ind_singular=False)
    
    Descripción:
            Función que devuelve la matriz Q y R de la factorización QR de una matriz A
    
    Parametros:
            A     np.ndarray que representa una matriz (mxn)
    
    Resultado:
            R     np.ndarray que representa matriz R (mxm) de la factorización A=QR
            Q     np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    
    Para obtener la factorización primero se calcula la matriz auxiliar Arv. La matriz Q 
    (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 

In [10]:
#Código "Q_j"
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j

    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para constru

In [11]:
#Ayuda "Q_j"
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(Arv, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [12]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [13]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [14]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(6,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv para visualizarla
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q,R=fQR.matriz_Q_R(A)
print('\n\nQ=\n',np.round(Q,4))
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))
print('\n\nA=\n',np.round(A,4))



A=
 [[-0.6378 -4.4442  2.6757 -2.498 ]
 [ 1.754  -3.5285  0.3085  0.7997]
 [ 3.7494  3.7053  1.8755 -4.9865]
 [ 5.6716  1.7626 -2.2987  4.2505]
 [-1.4752 -3.4578  3.2609 -1.2582]
 [-2.5241  2.9867 -3.0012 -1.761 ]]


Arv=
 [[ 7.6325  2.3711 -0.5772  1.9269]
 [-0.2121  8.0182 -3.6108 -1.0205]
 [-0.4534 -0.0474  4.7249 -5.7039]
 [-0.6858  0.2241  0.9519  4.1481]
 [ 0.1784  0.1726 -1.6304  1.3662]
 [ 0.3052 -0.39    1.8995 -1.1849]]


Q=
 [[-8.360e-02 -5.295e-01  1.514e-01 -4.855e-01 -6.688e-01  8.180e-02]
 [ 2.298e-01 -5.080e-01 -2.949e-01 -4.444e-01  6.055e-01 -1.950e-01]
 [ 4.912e-01  3.168e-01  6.991e-01 -3.911e-01  1.281e-01 -1.460e-02]
 [ 7.431e-01  1.000e-04 -3.957e-01  1.355e-01 -2.231e-01  4.724e-01]
 [-1.933e-01 -3.741e-01  3.806e-01  2.179e-01  3.363e-01  7.192e-01]
 [-3.307e-01  4.703e-01 -3.162e-01 -5.900e-01  8.230e-02  4.633e-01]]


R=
 [[ 7.6325  2.3711 -0.5772  1.9269]
 [ 0.      8.0182 -3.6108 -1.0205]
 [-0.      0.      4.7249 -5.7039]
 [ 0.     -0.      0.      4.148

In [15]:
#Comparemos con el código de numpy
Q_np, R_np=np.linalg.qr(A)
#notamos que numpy obtiene con signos contrarios Q y R
print('Q_np=\n',np.round(Q_np,4))
#Además R no guarda toda la matriz R de mxn, sino sólo la parte superior de nxn
print('\nR_np=\n',np.round(R_np,4))
#la multiplicación da por resultado A
print('\nQ_np@R_np=\n',np.round(Q_np@R_np,4))

Q_np=
 [[-8.360e-02 -5.295e-01 -1.514e-01 -4.855e-01]
 [ 2.298e-01 -5.080e-01  2.949e-01 -4.444e-01]
 [ 4.912e-01  3.168e-01 -6.991e-01 -3.911e-01]
 [ 7.431e-01  1.000e-04  3.957e-01  1.355e-01]
 [-1.933e-01 -3.741e-01 -3.806e-01  2.179e-01]
 [-3.307e-01  4.703e-01  3.162e-01 -5.900e-01]]

R_np=
 [[ 7.6325  2.3711 -0.5772  1.9269]
 [ 0.      8.0182 -3.6108 -1.0205]
 [ 0.      0.     -4.7249  5.7039]
 [ 0.      0.      0.      4.1481]]

Q_np@R_np=
 [[-0.6378 -4.4442  2.6757 -2.498 ]
 [ 1.754  -3.5285  0.3085  0.7997]
 [ 3.7494  3.7053  1.8755 -4.9865]
 [ 5.6716  1.7626 -2.2987  4.2505]
 [-1.4752 -3.4578  3.2609 -1.2582]
 [-2.5241  2.9867 -3.0012 -1.761 ]]


In [16]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[-8.360e-02, -5.295e-01,  1.514e-01, -4.855e-01, -6.688e-01,
         8.180e-02],
       [ 2.298e-01, -5.080e-01, -2.949e-01, -4.444e-01,  6.055e-01,
        -1.950e-01],
       [ 4.912e-01,  3.168e-01,  6.991e-01, -3.911e-01,  1.281e-01,
        -1.460e-02],
       [ 7.431e-01,  1.000e-04, -3.957e-01,  1.355e-01, -2.231e-01,
         4.724e-01],
       [-1.933e-01, -3.741e-01,  3.806e-01,  2.179e-01,  3.363e-01,
         7.192e-01],
       [-3.307e-01,  4.703e-01, -3.162e-01, -5.900e-01,  8.230e-02,
         4.633e-01]])

In [17]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1. -0.  0. -0. -0.  0.]
 [-0.  1.  0.  0.  0. -0.]
 [ 0.  0.  1.  0.  0. -0.]
 [-0.  0.  0.  1.  0. -0.]
 [-0.  0.  0.  0.  1. -0.]
 [ 0. -0. -0. -0. -0.  1.]]


In [18]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,5,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\nQ',j+1,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))
print('\nQ@R:\n',np.round(Q@R,4))
print('\nA:\n',np.round(A,4))



A=
 [[-2.  0.  4.  1.  2.]
 [ 5. -5. -3.  0.  5.]
 [-4. -2.  4.  4.  2.]
 [-2.  4.  4. -2.  3.]
 [-4. -1.  5.  0. -1.]]

Q 1 :
 [[-0.2481  0.6202 -0.4961 -0.2481 -0.4961]
 [ 0.6202  0.6918  0.2465  0.1233  0.2465]
 [-0.4961  0.2465  0.8028 -0.0986 -0.1972]
 [-0.2481  0.1233 -0.0986  0.9507 -0.0986]
 [-0.4961  0.2465 -0.1972 -0.0986  0.8028]]

 Q1 @ A=
 [[ 8.0623 -2.6047 -8.3103 -1.7365  1.3644]
 [-0.     -3.7057  3.1171  1.3598  5.3158]
 [ 0.     -3.0354 -0.8937  2.9122  1.7473]
 [ 0.      3.4823  1.5532 -2.5439  2.8737]
 [ 0.     -2.0354  0.1063 -1.0878 -1.2527]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.5918 -0.4847  0.5561 -0.325 ]
 [ 0.     -0.4847  0.8524  0.1693 -0.099 ]
 [ 0.      0.5561  0.1693  0.8057  0.1136]
 [ 0.     -0.325  -0.099   0.1136  0.9336]]

 Q2 @ Q1 @ A=
 [[ 8.0623 -2.6047 -8.3103 -1.7365  1.3644]
 [ 0.      6.2622 -0.5822 -3.2773 -1.9875]
 [ 0.     -0.     -2.0202  1.5001 -0.4767]
 [-0.     -0.      2.8455 -0.924   5.4251]
 [ 0.      0.  

In [19]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 6.65248741e+01, -9.00573138e-01,  8.27226468e+00, ...,
         7.43492421e+00, -8.25024991e+00, -7.75107619e+00],
       [-4.60946216e-02,  6.33833345e+01,  5.19682651e+00, ...,
        -8.15871727e+00,  4.92450634e+00,  6.48509446e+00],
       [-9.72496409e-02,  2.48863136e-03,  6.46838977e+01, ...,
         4.17759519e+00,  4.47186605e+00,  5.59202806e+00],
       ...,
       [ 1.75437551e-02,  6.78062070e-02, -4.33809833e-02, ...,
         6.94215764e-01, -2.42830450e-01, -1.41427551e-01],
       [ 1.01417993e-01,  2.00067757e-02,  9.82252017e-02, ...,
        -7.64606973e-02, -2.44234452e-02, -2.54484591e-01],
       [-1.36919769e-01,  3.50368921e-02,  7.72694635e-02, ...,
         1.15536532e-01,  1.32836354e-01, -1.70735510e-02]])

In [20]:
#Se puede verificar si una matriz es singular mediante el determinante (si
#este es igual a cero). La propiedad de singularidad o no singularidad se
#preserva bajo las operaciones efectuadas a la matriz A para obtener R (QA=R)
#y podemos checar si esta matriz es singular si alguno de los elementos
#de su diagonal es igual a cero
#La función fQR.matriz_auxiliar_Arv, fQR.matriz_Q_R tienen un argumentos adiciona
#para que en caso de que se encuentre una matriz singular suspenda los cálculos

#Obtención de matriz Q y R de una matriz singular A
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype='d')
print('A\n',A)
Q,R=fQR.matriz_Q_R(A)
print('\nQ:\n',np.round(Q,2))
#Nótese como existe un cero en la diagonal de esta matriz
print('\nR:\n',np.round(R,2))
print('\nQ@R:\n',Q@R)

#Al momento de calcular la factorización QR podemos solicitar que nos indique
#si la matriz es singular y suspenda los cálculos (regresando valores None)
fQR.matriz_Q_R(A,1)

A
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

Q:
 [[ 0.12  0.9   0.41]
 [ 0.49  0.3  -0.82]
 [ 0.86 -0.3   0.41]]

R:
 [[ 8.12  9.6  11.08]
 [-0.    0.9   1.81]
 [-0.    0.    0.  ]]

Q@R:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


(None, None)

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [21]:
#Código Solucion_SEL_QR_nxn
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones 

In [22]:
#Ayuda Solucion_SEL_QR_nxn
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con 
        n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
               elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)
    
    El Sistema de Ecuaciones Lineales establecido como Ax=b, es resuelto con esta función
    mediante la factorización QR como a continuación se explica:
    
      Si A*x = b y A = Q*R, con A matriz (mxn), b (mx1) y Q una matriz ortogonal (mxm) y 
                                R (mxn) una matriz triangular superior
      entonces Q*R*x = b.
      
      Aprovechando la propiedad de Q^(-1)=Q, es decir, Q^(-1)*Q=Ide

## Eliminación por bloques

### Funciones

In [23]:
#Código "crear_bloques"
lines=inspect.getsource(fQR.crear_bloques)
print(lines)

def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A deb

In [24]:
#Ayuda "crear_bloques"
help(fQR.crear_bloques)

Help on function crear_bloques in module funciones_factorizacion_QR:

crear_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que divide la matriz A(mxn) y el vector b(mx1) en cuatro submatrices y dos vectores respectivamente
    
    Parámetros:                            
      A     Matriz (mxn) 
      b     vector (mx1)
      m1    Número de renglones del los bloques superiores
      n1    Número de columnas de los bloques de la izquierda
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
      b es un parámetro opcional, si no es introducido solo regresa las submatrices de A
    
    
    Resultado:
      A11   numpy.ndarray que representa la matriz de dimensión m1xn1
      A12   numpy.ndarray que representa la matriz de dimensión m2xn1
      A12   numpy.ndarray que representa la matriz de dimensión m1xn2
      A22   numpy.ndarray que representa la matriz 

In [25]:
#Código "eliminacion_bloques"
lines=inspect.getsource(fQR.eliminacion_bloques)
print(lines)

def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:


In [26]:
#Ayuda "eliminacion_bloques"
help(fQR.eliminacion_bloques)

Help on function eliminacion_bloques in module funciones_factorizacion_QR:

eliminacion_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que usa la eliminación por bloques para resolver un sistema de ecuaciones Ax=b.
    
    Parámetros:                            
      A     numpy.ndarray que representa la matriz (mxn) que representan loc coeficientes del sistema de      ecuaciones
      b     numpy.ndarray que representa el vector (mx1) que representan el lado derecho del sistema de       ecuaciones
      m1    Número de renglones del los bloques superiores del subconjunto a resolver
      n1    Número de columnas de los bloques del subconjunto a resolver
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
    
    Resultado:
      x   numpy.ndarray que representa la solución al sistema de ecuaciones Ax=b
    
    Este método consiste en eliminar un subconju

## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [27]:
#Generamos un sistema de ecuaciones lineales
m=12
n=12
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-4.89  0.48 -4.19 -5.1   3.38  1.64 -4.74  0.66 -5.83  4.06  3.66  5.42]
 [-4.03  4.6  -3.78  2.85  2.81 -0.91  2.81  5.41 -3.92 -4.1   3.24 -1.66]
 [-1.73  2.24  2.89 -3.59  4.02  1.58  5.98  0.79 -1.85 -1.19 -1.46  3.75]
 [-1.86 -1.07  2.23  5.94  2.6  -2.64  5.48  0.09 -5.28  3.61  3.56 -2.21]
 [ 4.1  -3.72 -3.74  2.59 -1.27 -2.38  0.3   5.57 -4.9  -4.58  5.25  2.66]
 [-3.66  2.84 -2.16 -4.35  4.83 -0.8  -2.55 -0.71  1.26 -1.54  1.23  4.67]
 [ 3.31  4.82  2.84  2.72  1.68  4.95  0.16  2.39  0.34  5.14  4.06  4.03]
 [ 5.07  5.1  -1.52  2.    4.16  2.15 -3.56 -5.51  2.29 -2.2   2.07  3.66]
 [-5.35 -2.97 -3.87  4.58 -5.16  0.95 -1.66  2.21  3.29 -0.11  1.42 -1.78]
 [-1.11  2.85  0.76  2.1   4.12 -3.29  1.27 -5.05 -1.89 -0.86  3.1  -1.35]
 [-3.55  2.61  5.1   5.73  0.16 -5.09 -3.72  5.4  -3.3  -1.13  3.65 -3.84]
 [-5.1  -5.42  0.27 -5.55  0.18  1.15 -2.97 -4.99  0.04  0.26 -4.94 -1.81]]
b: [[ 2.35]
 [-1.2 ]
 [ 5.  ]
 [ 0.12]
 [ 5.76]
 [-0.24]
 [ 3.54]
 [-3.  ]
 [ 2.85]
 [-0.82]
 [ 

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [28]:
np.linalg.solve(A,b)

array([[-0.62959702],
       [-0.43238955],
       [ 1.37877071],
       [-0.31685579],
       [-0.93029216],
       [ 0.83148625],
       [ 0.06228694],
       [-0.20088111],
       [-0.31516119],
       [-0.98617982],
       [ 1.33572072],
       [ 0.5767964 ]])

Ahora usemos la factorización QR

In [29]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[-0.62959702],
       [-0.43238955],
       [ 1.37877071],
       [-0.31685579],
       [-0.93029216],
       [ 0.83148625],
       [ 0.06228694],
       [-0.20088111],
       [-0.31516119],
       [-0.98617982],
       [ 1.33572072],
       [ 0.5767964 ]])

In [30]:
x=fQR.eliminacion_bloques(A,b)
x

array([[-0.62959702],
       [-0.43238955],
       [ 1.37877071],
       [-0.31685579],
       [-0.93029216],
       [ 0.83148625],
       [ 0.06228694],
       [-0.20088111],
       [-0.31516119],
       [-0.98617982],
       [ 1.33572072],
       [ 0.5767964 ]])

In [31]:
#Se prueba con un sistema más grande
m=10**2
n=10**2
A=np.round(fQR.crear_matriz_aleatoria(m,n,10,-10,False),2)
b=np.round(fQR.crear_matriz_aleatoria(m,1,10,-10,False),2)
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[-1.62594071e-01],
       [-9.40220693e-02],
       [ 7.10054526e-02],
       [ 4.85183155e-01],
       [ 2.61990396e-02],
       [ 2.08921346e-01],
       [ 9.95058973e-02],
       [-6.01563433e-02],
       [-1.29522727e+00],
       [ 4.14385324e-01],
       [ 9.17471745e-01],
       [-9.02331528e-01],
       [ 5.01253779e-01],
       [ 6.53200567e-01],
       [-9.18711146e-01],
       [ 3.81650586e-02],
       [ 3.01993775e-01],
       [-6.36293858e-02],
       [ 8.15017925e-01],
       [-6.93155476e-01],
       [-4.34257201e-01],
       [-6.41935953e-02],
       [-8.08850466e-01],
       [ 3.05408649e-01],
       [-4.29177330e-01],
       [ 4.98362732e-01],
       [-2.49558090e-01],
       [ 1.59640360e-01],
       [-8.21404965e-01],
       [ 5.21401175e-01],
       [ 2.09243626e-01],
       [ 7.72897285e-02],
       [-9.96846474e-01],
       [ 1.10421234e+00],
       [ 1.17749108e+00],
       [ 6.28751177e-01],
       [-3.42771977e-01],
       [-6.06242857e-01],
       [-7.5

Por último usemos eliminación por bloques con QR

In [32]:
x=fQR.eliminacion_bloques(A,b)
x

array([[-1.62594071e-01],
       [-9.40220693e-02],
       [ 7.10054526e-02],
       [ 4.85183155e-01],
       [ 2.61990396e-02],
       [ 2.08921346e-01],
       [ 9.95058973e-02],
       [-6.01563433e-02],
       [-1.29522727e+00],
       [ 4.14385324e-01],
       [ 9.17471745e-01],
       [-9.02331528e-01],
       [ 5.01253779e-01],
       [ 6.53200567e-01],
       [-9.18711146e-01],
       [ 3.81650586e-02],
       [ 3.01993775e-01],
       [-6.36293858e-02],
       [ 8.15017925e-01],
       [-6.93155476e-01],
       [-4.34257201e-01],
       [-6.41935953e-02],
       [-8.08850466e-01],
       [ 3.05408649e-01],
       [-4.29177330e-01],
       [ 4.98362732e-01],
       [-2.49558090e-01],
       [ 1.59640360e-01],
       [-8.21404965e-01],
       [ 5.21401175e-01],
       [ 2.09243626e-01],
       [ 7.72897285e-02],
       [-9.96846474e-01],
       [ 1.10421234e+00],
       [ 1.17749108e+00],
       [ 6.28751177e-01],
       [-3.42771977e-01],
       [-6.06242857e-01],
       [-7.5

In [33]:
#Se prueba con un sistema más grande
m=10**3
n=10**3
A=np.round(fQR.crear_matriz_aleatoria(m,n,10,-10,False),2)
b=np.round(fQR.crear_matriz_aleatoria(m,1,10,-10,False),2)

In [34]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[ 5.00489015e+00],
       [-1.07275014e+00],
       [ 6.31092507e-01],
       [-9.06073997e-03],
       [ 1.68537445e+00],
       [-2.24321163e+00],
       [ 1.20400936e+00],
       [ 2.77863530e+00],
       [ 4.85727797e-02],
       [-3.10932293e+00],
       [ 3.42332070e-01],
       [-2.49755404e+00],
       [-1.21442559e+00],
       [ 4.73107893e-01],
       [-6.44618577e-01],
       [-1.23395033e-01],
       [-2.51325290e+00],
       [-2.21775177e+00],
       [ 3.39762777e-01],
       [ 1.03052143e+00],
       [-2.01152996e+00],
       [-1.39021469e-01],
       [ 4.94515783e-01],
       [-7.36859288e-02],
       [-1.45107155e+00],
       [ 6.72923459e-01],
       [ 4.50156838e-01],
       [ 1.11080132e+00],
       [ 1.66832189e+00],
       [-8.34255506e-01],
       [-1.39503798e+00],
       [ 3.86386530e+00],
       [ 1.34395577e+00],
       [-6.15769449e-01],
       [ 6.63187660e-01],
       [ 1.33652061e-01],
       [ 3.66210311e+00],
       [-4.34662882e-01],
       [-1.3

In [35]:
x=fQR.eliminacion_bloques(A,b)
x

KeyboardInterrupt: 

### B) Sistema sin solución

In [ ]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [5]], dtype='d')
print("A:",A)
print("b:",b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la eliminación por bloques usando la factorización QR

In [ ]:
fQR.eliminacion_bloques(A,b)

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [ ]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [0]], dtype='d')
print("A:",A)
print("b:",b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos eliminación por bloques usando la factorización QR

In [ ]:
fQR.eliminacion_bloques(A,b)